In [1]:
import os
from dataclasses import dataclass
from pathlib import Path
import urllib.request as request
import gdown
import zipfile
from pneuDetection import logger
from pneuDetection.constants import *
from pneuDetection.utils.common import read_yaml, create_directories, get_size

In [2]:
%pwd

'c:\\Users\\cisco\\Python_Projects\\ML+AI\\Pneumonia-Detection\\research'

In [4]:
os.chdir("../")
%pwd

'c:\\Users\\cisco\\Python_Projects\\ML+AI\\Pneumonia-Detection'

In [5]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path         # config.yaml - data_ingestion
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,  # src/pneuDetection/constants/__init__.py >> config.yaml
        params_filepath = PARAMS_FILE_PATH): # src/pneuDetection/constants/__init__.py >> params.yaml

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) # creating artifacts folder


    def get_data_ingestion_config(self) -> DataIngestionConfig: # DataIngestionConfig type: root_dir, source_URL, local_data_file, unzip_dir
        config = self.config.data_ingestion # extracting data_ingestion from config.yaml

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig): # config via ConfigurationManager
        self.config = config


    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            prefix = 'https://drive.google.com/uc?/export=download&id=' # GoogleDrive standard prefix (valid as of 4/24/2024)
            dataset_url = self.config.source_URL # config.yaml: data_ingestion >> source_URL
            logger.info(f"Standard Google Drive prefix and dataset_url set.")
            
            file_id = dataset_url.split("/")[-2] # splitting unique key
            logger.info(f"Extracted file ID: {file_id}")
            
            zip_download_dir = self.config.local_data_file # config.yaml: data_ingestion >> local_data_file
            logger.info(f"Zip download dir set: {zip_download_dir}")
            
            os.makedirs("artifacts/data_ingestion", exist_ok=True) # creating artifacts folders
            logger.info(f"End local artifacts/data_ingestion creation.")
            
            logger.info(f"Starting data download from {dataset_url} to {zip_download_dir} file")
            logger.info(f"...gdown.download({prefix} + {file_id}, {zip_download_dir})...")
            gdown.download(prefix + file_id, zip_download_dir) 
            logger.info(f"Completed data download from {dataset_url} to {zip_download_dir} file")

        except Exception as e:
            raise


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        logger.info(f"Start DataIngestion.extract_zip_file")
        
        logger.info(f"Defining unzip_path...")
        unzip_path = self.config.unzip_dir
        logger.info(f"unzip_path: {unzip_path}")
        
        logger.info(f"Creating folder for unzipped data...")
        os.makedirs(unzip_path, exist_ok=True)
        logger.info(f"Complete folder creation for unzipped data: {unzip_path}")
        
        logger.info(f"Starting file extraction...")
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Completed zip extraction.")
        
        logger.info(f"End DataIngestion.extract_zip_file.")

## Pipeline

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file() 
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-23 15:26:17,771: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-23 15:26:17,772: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-23 15:26:17,774: INFO: common: created directory at: artifacts]
[2024-04-23 15:26:17,775: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-23 15:26:17,776: INFO: 3162374175: Downloading data from https://drive.google.com/file/d/1WI9Axo6yRc2ayIemo-hLWpZuKlBb2ydH/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1WI9Axo6yRc2ayIemo-hLWpZuKlBb2ydH
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1WI9Axo6yRc2ayIemo-hLWpZuKlBb2ydH&confirm=t&uuid=0679840d-11f6-447e-a255-ddc4e8f496a8
To: c:\Users\cisco\Python_Projects\ML+AI\Pneumonia-Detection\artifacts\data_ingestion\data.zip
100%|██████████| 1.15G/1.15G [21:28<00:00, 891kB/s] 

[2024-04-23 15:47:48,670: INFO: 3162374175: Downloaded data from https://drive.google.com/file/d/1WI9Axo6yRc2ayIemo-hLWpZuKlBb2ydH/view?usp=sharing into file artifacts/data_ingestion/data.zip]
